In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Breast_Cancer"
cohort = "GSE236725"

# Input paths
in_trait_dir = "../../input/GEO/Breast_Cancer"
in_cohort_dir = "../../input/GEO/Breast_Cancer/GSE236725"

# Output paths
out_data_file = "../../output/preprocess/Breast_Cancer/GSE236725.csv"
out_gene_data_file = "../../output/preprocess/Breast_Cancer/gene_data/GSE236725.csv"
out_clinical_data_file = "../../output/preprocess/Breast_Cancer/clinical_data/GSE236725.csv"
json_path = "../../output/preprocess/Breast_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The 31 pairs of FF and FFPE data for the development of 72GC for FFPE in 2014"
!Series_summary	"The study aimed to develop the 72GC (the 72-gene classifier) for recurrence-risk prediction for patients with estrogen receptor positive and node-negative breast cancer. 72-GC could differentiate the high-risk from the low-risk patients with a high statistical significance, and is considered to be applicable to formalin-fixed, paraffin embedded (FFPE) tumor tissues because the results of 72-GC on FF (fresh-frozen) tissues and FFPE tissues showed a high concordance.  J06 31 pairs of FF (fresh‑frozen) and FFPE data are included in the concordance analysis of 72GC high/low results between FF and FFPE (Table 3). A long time passed, and now it is unclear how these 31 cases were distributed among the analysis.    *Note: This old data has been updated multiple times by the other members.  Then, there are some differences from the original paper and unclear poi

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# According to the background information, this dataset contains gene expression data from Affymetrix microarrays
is_gene_available = True

# 2.1 Data Availability
# For trait: Looking at the sample characteristics, distant.recurrence (key 1) indicates breast cancer recurrence
trait_row = 1  # 'distant.recurrence: Yes/No' is our target trait

# For age: There is no age information available in the sample characteristics
age_row = None

# For gender: No gender information is available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert breast cancer recurrence status to binary."""
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if value.lower() == 'yes':
        return 1  # Recurrence present
    elif value.lower() == 'no':
        return 0  # No recurrence
    return None

def convert_age(value):
    """Convert age to continuous values.
    Not used in this dataset as age information is unavailable."""
    return None

def convert_gender(value):
    """Convert gender to binary (0=female, 1=male).
    Not used in this dataset as gender information is unavailable."""
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                             is_gene_available=is_gene_available, 
                             is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # We'll need to use the existing clinical data that has been loaded in a previous step
    # Find available files in the cohort directory
    available_files = os.listdir(in_cohort_dir)
    print(f"Available files in {in_cohort_dir}: {available_files}")
    
    # Check for different possible clinical data files
    clinical_file_name = None
    for file in available_files:
        if 'clinical' in file.lower() or 'characteristics' in file.lower() or 'phenotype' in file.lower():
            clinical_file_name = file
            break
    
    if clinical_file_name:
        # If we found a clinical data file, load it
        clinical_data = pd.read_csv(os.path.join(in_cohort_dir, clinical_file_name))
        print(f"Loaded clinical data from {clinical_file_name}")
    else:
        # If no clinical data file is found, we'll create a DataFrame from the sample characteristics dictionary
        # that was provided in the previous step's output
        print("No clinical data file found. Creating DataFrame from sample characteristics dictionary.")
        
        # Since we don't directly have the sample_characteristics variable, we need to create a representative DataFrame
        # from what we can infer from the sample characteristics dictionary shown in the previous output
        
        # Create a sample DataFrame based on the sample characteristics dictionary
        # We'll assume the format matches what geo_select_clinical_features expects
        sample_characteristics = {}
        
        # From the dictionary in the output, key 1 contains distant.recurrence data
        sample_characteristics[1] = ['distant.recurrence: Yes', 'distant.recurrence: No']
        
        # Create a dataframe with columns matching what's expected by geo_select_clinical_features
        clinical_data = pd.DataFrame(index=range(len(sample_characteristics[1])), columns=['TITLES'])
        clinical_data.iloc[:, 0] = sample_characteristics[1]
    
    # Extract the clinical features using the function from the library
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    print("Preview of extracted clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Save the clinical data to the specified path
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to: {out_clinical_data_file}")


Available files in ../../input/GEO/Breast_Cancer/GSE236725: ['GSE236725_family.soft.gz', 'GSE236725_series_matrix.txt.gz']
No clinical data file found. Creating DataFrame from sample characteristics dictionary.
Preview of extracted clinical features:
{'TITLES': [0.0]}
Clinical data saved to: ../../output/preprocess/Breast_Cancer/clinical_data/GSE236725.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/Breast_Cancer/GSE236725/GSE236725_family.soft.gz
Matrix file: ../../input/GEO/Breast_Cancer/GSE236725/GSE236725_series_matrix.txt.gz
Found the matrix table marker at line 67


Gene data shape: (54675, 62)
First 20 gene/probe identifiers:
['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at', '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at', '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at', '1552263_at', '1552264_a_at', '1552266_at']


### Step 4: Gene Identifier Review

In [5]:
# The identifiers shown (1007_s_at, 1053_at, etc.) are Affymetrix probe IDs, 
# not human gene symbols. These are identifiers from Affymetrix microarray platforms
# and need to be mapped to standard gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=3))

# Looking at the output, it appears the gene symbols are in the 'Gene Symbol' column
# and the probe IDs are in the 'ID' column
print("\nExamining ID and Gene Symbol columns format (first 3 rows):")
if 'ID' in gene_annotation.columns and 'Gene Symbol' in gene_annotation.columns:
    for i in range(min(3, len(gene_annotation))):
        print(f"Row {i}: ID={gene_annotation['ID'].iloc[i]}, Gene Symbol={gene_annotation['Gene Symbol'].iloc[i]}")

    # Check the quality and completeness of the mapping
    non_null_symbols = gene_annotation['Gene Symbol'].notna().sum()
    total_rows = len(gene_annotation)
    print(f"\nGene Symbol column completeness: {non_null_symbols}/{total_rows} rows ({non_null_symbols/total_rows:.2%})")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757'], 'SPOT_ID': [nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns contain gene identifiers and gene symbols
# From our analysis, we can see that:
# - The 'ID' column in the gene annotation contains Affymetrix probe IDs (e.g., "1007_s_at")
# - The 'Gene Symbol' column contains the human gene symbols (e.g., "DDR1 /// MIR4640")

# These match what we need for mapping:
prob_col = 'ID'          # The column with probe IDs that match our expression data index
gene_col = 'Gene Symbol' # The column with gene symbols we want to map to

# 2. Get a gene mapping dataframe using the library function
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("Preview of mapping dataframe:")
print(preview_df(mapping_df, n=5))

# 3. Convert probe-level expression to gene-level expression
# Extract the gene expression data using the function from the library
gene_expr_df = get_genetic_data(matrix_file)
print(f"Original gene expression data shape: {gene_expr_df.shape}")

# Apply the gene mapping to convert probe-level to gene-level expression
gene_data = apply_gene_mapping(gene_expr_df, mapping_df)
print(f"Mapped gene expression data shape: {gene_data.shape}")
print("Preview of mapped gene expression data:")
print(preview_df(gene_data, n=3))

# Save the gene expression data to the specified output path
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


Gene mapping dataframe shape: (45782, 2)
Preview of mapping dataframe:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}


Original gene expression data shape: (54675, 62)
Mapped gene expression data shape: (21278, 62)
Preview of mapped gene expression data:
{'GSM7574510': [8.256737349, 6.911184187, 8.633719548], 'GSM7574511': [5.758196709, 4.392330888, 7.459701646], 'GSM7574512': [7.90550267, 7.006615101, 10.748925159], 'GSM7574513': [7.621566619, 5.523093236, 10.236430428], 'GSM7574514': [7.746445504, 7.535800803, 12.058622905], 'GSM7574515': [8.310367085, 7.079364514, 11.198075410000001], 'GSM7574516': [7.775885539, 6.449054113, 7.317667267], 'GSM7574517': [7.525905291, 4.108699329, 11.644830382999999], 'GSM7574518': [7.849060138, 6.652675857, 10.843219882], 'GSM7574519': [8.408982277, 7.172603935, 8.447698456000001], 'GSM7574520': [7.445540368, 7.694966106, 7.320610432], 'GSM7574521': [8.128703262, 6.623905779, 10.761083019], 'GSM7574522': [7.667458, 7.469764175, 7.452024023], 'GSM7574523': [8.775423107, 6.766474906, 7.213160362], 'GSM7574524': [8.11916547, 6.245697206, 8.056183743], 'GSM7574525': [8.1

Gene expression data saved to: ../../output/preprocess/Breast_Cancer/gene_data/GSE236725.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data - respecting the analysis from Step 2
# From Step 2, we determined:
# trait_row = None  # No Breast Cancer subtype data available
# age_row = 2
# gender_row = None
is_trait_available = trait_row is not None

# Skip clinical feature extraction when trait_row is None
if is_trait_available:
    try:
        # Load the clinical data from file
        soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        background_info, clinical_data = get_background_and_clinical_data(matrix_file)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            gender_row=gender_row,
            convert_gender=convert_gender,
            age_row=age_row,
            convert_age=convert_age
        )
        
        print(f"Extracted clinical data shape: {clinical_features.shape}")
        print("Preview of clinical data (first 5 samples):")
        print(clinical_features.iloc[:, :5])
        
        # Save the properly extracted clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical data: {e}")
        is_trait_available = False
else:
    print(f"No trait data ({trait}) available in this dataset based on previous analysis.")

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from triple negative breast cancer vs. luminal tumors, but no explicit breast cancer subtype labels in the sample characteristics."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # Create an empty DataFrame for metadata purposes
    empty_df = pd.DataFrame()
    
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=empty_df,  # Empty dataframe for metadata
        note="Dataset contains gene expression data from triple negative breast cancer vs. luminal tumors, but no explicit breast cancer subtype labels in the sample characteristics."
    )

Gene data shape before normalization: (21278, 62)
Gene data shape after normalization: (19845, 62)


Normalized gene data saved to ../../output/preprocess/Breast_Cancer/gene_data/GSE236725.csv


Extracted clinical data shape: (1, 62)
Preview of clinical data (first 5 samples):
               GSM7574510  GSM7574511  GSM7574512  GSM7574513  GSM7574514
Breast_Cancer         1.0         1.0         1.0         1.0         0.0
Clinical data saved to ../../output/preprocess/Breast_Cancer/clinical_data/GSE236725.csv
Gene data columns (first 5): ['GSM7574510', 'GSM7574511', 'GSM7574512', 'GSM7574513', 'GSM7574514']
Clinical data columns (first 5): ['GSM7574510', 'GSM7574511', 'GSM7574512', 'GSM7574513', 'GSM7574514']
Found 62 common samples between gene and clinical data
Initial linked data shape: (62, 19846)
Preview of linked data (first 5 rows, first 5 columns):
            Breast_Cancer      A1BG  A1BG-AS1       A1CF        A2M
GSM7574510            1.0  8.256737  6.911184   8.633720  19.862964
GSM7574511            1.0  5.758197  4.392331   7.459702  19.472159
GSM7574512            1.0  7.905503  7.006615  10.748925  21.226190
GSM7574513            1.0  7.621567  5.523093  10.2364

Linked data shape after handling missing values: (62, 19846)
For the feature 'Breast_Cancer', the least common label is '1.0' with 16 occurrences. This represents 25.81% of the dataset.
The distribution of the feature 'Breast_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Breast_Cancer/GSE236725.csv
